In [86]:
import numpy as np
import pandas as pd

### 1. Load the dataset to a data frame.

In [87]:
data_df = pd.read_csv("../data/mtsamples_descriptions_clean.csv")

### 2. Check the top rows.

In [88]:
data_df.head()

,id,id_description,medical_specialty_new,text,year,borough
0,0,0,Gastroenterology,EGD with photos and biopsies,2013.0,Merton
1,1,0,Gastroenterology,This is a 75-year-old female who presents wit...,2013.0,Merton
2,2,0,Gastroenterology,She has a previous history of hiatal hernia,2013.0,Merton
3,3,0,Gastroenterology,She was on Prevacid currently,2013.0,Merton
4,4,1,Urology,"Pelvic tumor, cystocele, rectocele, and uteri...",2013.0,Harrow


### 3. Check for duplicate rows in the id column.

In [89]:
data_df.sort_values("id")

,id,id_description,medical_specialty_new,text,year,borough
0,0,0,Gastroenterology,EGD with photos and biopsies,2013.0,Merton
1,1,0,Gastroenterology,This is a 75-year-old female who presents wit...,2013.0,Merton
2,2,0,Gastroenterology,She has a previous history of hiatal hernia,2013.0,Merton
3,3,0,Gastroenterology,She was on Prevacid currently,2013.0,Merton
4,4,1,Urology,"Pelvic tumor, cystocele, rectocele, and uteri...",2013.0,Harrow
...,...,...,...,...,...,...
3243,3243,2356,Ophthalmology,Patient referred for narrow angles and possib...,2010.0,Hounslow
3244,3244,2357,Dentistry,Letter on evaluation regarding extraction of ...,2016.0,City of London
3245,3245,2358,Neurology,Patient seen in Neuro-Oncology Clinic because...,2010.0,Kensington and Chelsea
3246,3246,2359,Gastroenterology,Common description of EGD.,2016.0,Newham


In [90]:
data_df["id"].duplicated().head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: id, dtype: bool

In [91]:
data_df[data_df["id"].duplicated()].shape[0]

0

There are no duplicated values in the "id" column. They are unique.

### 4. Check how many rows in each column have missing values.

In [92]:
#Just in case there is something resulting from a division by 0 or similar.
pd.options.mode.use_inf_as_na = True

In [93]:
print("Total number of rows: %d." % (data_df.shape[0]))
for col in data_df:
    N_missing = data_df[data_df[col].isna()].shape[0]
    print("Column '%s' has %d missing values." % (col, N_missing))

Total number of rows: 3248.
Column 'id' has 0 missing values.
Column 'id_description' has 0 missing values.
Column 'medical_specialty_new' has 1 missing values.
Column 'text' has 1 missing values.
Column 'year' has 1 missing values.
Column 'borough' has 1 missing values.


### 5. Remove the rows with missing values.

In [94]:
nonmiss_df = data_df.dropna()

In [95]:
print("Total number of rows: %d." % (nonmiss_df.shape[0]))
for col in data_df:
    N_missing = nonmiss_df[nonmiss_df[col].isna()].shape[0]
    print("Column '%s' has %d missing values." % (col, N_missing))

Total number of rows: 3245.
Column 'id' has 0 missing values.
Column 'id_description' has 0 missing values.
Column 'medical_specialty_new' has 0 missing values.
Column 'text' has 0 missing values.
Column 'year' has 0 missing values.
Column 'borough' has 0 missing values.


### 6. See if there are weird values in the 'year' column

In [96]:
nonmiss_df["year"].describe()

count    3245.000000
mean     2013.075809
std         2.405639
min      2010.000000
25%      2010.000000
50%      2013.000000
75%      2016.000000
max      2016.000000
Name: year, dtype: float64

year values range from 2010 to 2016, which are valid values.

In [97]:
nonmiss_df["year"][nonmiss_df["year"].apply(lambda x: x-np.floor(x) != 0)].count()

0

All year values are integer.

### 7. Show the 10 most frequent values in 'medical_speciality_new'

In [98]:
nonmiss_df["medical_specialty_new"].value_counts().head(10)

 Orthopedic                    475
 Radiology                     366
 Cardiovascular / Pulmonary    355
 Gastroenterology              333
 Obstetrics / Gynecology       233
 Urology                       228
 General Medicine              197
 Ophthalmology                 129
 ENT - Otolaryngology          124
 Surgery                        92
Name: medical_specialty_new, dtype: int64

These are the values we'll use for modeling.

### 8. Show the 10 most frequent values in 'text' column

In [99]:
nonmiss_df["text"].value_counts().head(10)

 Laparoscopic appendectomy                                       7
 Colonoscopy                                                     5
 Laparoscopic cholecystectomy                                    5
 Cataract, right eye                                             5
 Normal review of systems template                               4
 Circumcision                                                    3
 Sample progress note - Gen Med.                                 3
 Normal nuclear myocardial perfusion scan.                       3
 An example/template for a routine normal male physical exam.    3
 Normal physical exam template                                   3
Name: text, dtype: int64

These values will be used for natural language processing.

### 9. Add a'length' column with the length of the 'text'

In [102]:
final_df = nonmiss_df.copy()
final_df["length"] = nonmiss_df["text"].apply(lambda x: len(x))

In [103]:
final_df

,id,id_description,medical_specialty_new,text,year,borough,length
0,0,0,Gastroenterology,EGD with photos and biopsies,2013.0,Merton,29
1,1,0,Gastroenterology,This is a 75-year-old female who presents wit...,2013.0,Merton,106
2,2,0,Gastroenterology,She has a previous history of hiatal hernia,2013.0,Merton,44
3,3,0,Gastroenterology,She was on Prevacid currently,2013.0,Merton,30
4,4,1,Urology,"Pelvic tumor, cystocele, rectocele, and uteri...",2013.0,Harrow,56
...,...,...,...,...,...,...,...
3242,3242,2355,Ophthalmology,The patient was referred for evaluation of ca...,2010.0,Newham,65
3243,3243,2356,Ophthalmology,Patient referred for narrow angles and possib...,2010.0,Hounslow,69
3244,3244,2357,Dentistry,Letter on evaluation regarding extraction of ...,2016.0,City of London,85
3245,3245,2358,Neurology,Patient seen in Neuro-Oncology Clinic because...,2010.0,Kensington and Chelsea,124
